In [1]:
# all necessary requestcompletion imports 
import src.requestcompletion as rc
from src.requestcompletion.llm import MessageHistory, UserMessage, SystemMessage
from src.requestcompletion.visuals.agent_viewer import AgentViewer
from src.requestcompletion.nodes.library import from_function

<img src="./assets/Telus-Symbol.png" alt="Telus Symbol" width="200"/>

## RC Demo: Customer Service Ticketing Agent + Notion:

In this demo we will demonstrate how to use RequestCompletion to built AI Agents and combine them to perrform complex tasks using just a few lines of code.

### Step 1: Import and use Notion Agent from sample_agents

In [2]:
from demo.sample_agents import NotionAgent

### Step 2: Invoke the agent 🚀

In [ ]:
USER_MESSAGE = UserMessage(...)

In [ ]:
mess_hist = MessageHistory([USER_MESSAGE])

with rc.Runner() as runner:
    result = await runner.run(NotionAgent, mess_hist)

[+187.883s] RC.RUNNER   : INFO     - START CREATED Notion Agent - (user: Under the Telus Demo page, add a new page called 'RC is Awesome'. Inside that page write something awesome about RC., )
[+189.030s] RC.RUNNER   : INFO     - Notion Agent CREATED find_page Node - ({'query': 'Telus Demo'}, )
[+189.317s] RC.RUNNER   : INFO     - find_page Node DONE [PageProperties(id='1e01b3a3-98fd-80f5-91a1-f4ea96d868df', title='Telus Demo'), PageProperties(id='1d71b3a3-98fd-80e3-bfc4-c4226c281833', title='Agent Demo Root')]
[+190.558s] RC.RUNNER   : INFO     - Notion Agent CREATED create_page Node - ({'properties': {'parent_id': '1e01b3a3-98fd-80f5-91a1-f4ea96d868df', 'title': 'RC is Awesome'}}, )
[+190.792s] RC.RUNNER   : INFO     - create_page Node DONE 1e31b3a3-98fd-810b-9968-c7b33e2c5dde
[+193.760s] RC.RUNNER   : INFO     - Notion Agent CREATED add_block Node - ({'properties': {'content': 'RC is a fantastic platform that fosters creativity and collaboration. It empowers users to build and manag

we can also visualize the agent control flow

In [5]:
viewer = AgentViewer(stamps=result.all_stamps,
                     node_heap=result.node_heap,
                     request_heap=result.request_heap)

viewer.display_graph()

## Now lets build our own Agent for Telus

We have some mock functions that we will give the Agent access to as tools.

In [2]:
# imports
from demo.sample_tools.notion_tools import NotionAgenticTool
from demo.sample_tools.telus_tools import get_service_tickets, create_service_ticket, check_network_status, analyze_call_volume

In [3]:
# create randomized mock data (mock telus Database)
mock_entries = [
    ("internet_outage", "Vancouver, BC"),
    ("billing", "Vancouver, BC"),
    ("device_support", "Langley, BC"),
    ("internet_outage", "Burnbay, BC"),
    ("tv_service", "Vancouver, BC"),
    ("mobile_service", "Burnaby, BC"),
    ("billing", "Kelowna, BC"),
    ("fiber_installation", "Richmond, BC"),
    ("device_support", "Victoria, BC"),
    ("network_latency", "Abbotsford, BC")
]

# Populate the mock database
for issue_type, location in mock_entries:
    result = create_service_ticket(issue_type, location)

### Step 1: Prompts for the Agent

In [4]:
SYSTEM_PROMPT_TELUS = SystemMessage("You are a helpful Telus agent. You have access to the following tools: check_network_status," \
"analyze_call_volume, create_service_ticket, get_service_tickets, and NotionTool. Your goal is to resolve the user's request as efficiently as possible." \
"Today is April 28th 2025.")

In [5]:
USER_MESSAGE = UserMessage("Analyze the call volume and network status for the Vancouver and Burnaby area this week." \
               "Categorize the service tickets by issue type and create a summary page showing resolution times and identifying any recurring problems." \
               "Give me an analysis page and a summary page. The notion root page is 'Telus Demo'.")

### Step 2: Construct the Agent

In [6]:
TelusTicketingAgent = rc.library.tool_call_llm(connected_nodes={NotionAgenticTool,
                                                                from_function(get_service_tickets),   # make functions into nodes before passing as connecte nodes
                                                                from_function(check_network_status),
                                                                from_function(analyze_call_volume),
                                                                },
                                               model=rc.llm.OpenAILLM("gpt-4o"),
                                               pretty_name="Telus Agent",
                                               system_message=SYSTEM_PROMPT_TELUS)

### Step 3: Invoke the Agent 🚀

In [7]:
mess_hist = MessageHistory([USER_MESSAGE])

with rc.Runner(executor_config=rc.ExecutorConfig(timeout=50)) as runner:
    result = await runner.run(TelusTicketingAgent, mess_hist)

[+144.339s] RC.RUNNER   : INFO     - START CREATED Telus Agent - (user: Analyze the call volume and network status for the Vancouver and Burnaby area this week.Categorize the service tickets by issue type and create a summary page showing resolution times and identifying any recurring problems.Give me an analysis page and a summary page. The notion root page is 'Telus Demo'., )
[+146.467s] RC.RUNNER   : INFO     - Telus Agent CREATED analyze_call_volume Node - ({'time_range': 'weekly', 'region': 'BC'}, )
[+146.468s] RC.RUNNER   : INFO     - analyze_call_volume Node DONE {'average_wait_time_minutes': 9.1, 'peak_hours': ['12:00-13:00', '10:00-11:00']}
[+148.399s] RC.RUNNER   : INFO     - Telus Agent CREATED check_network_status Node - ({'location': 'Vancouver'}, )
[+148.402s] RC.RUNNER   : INFO     - check_network_status Node DONE {'latency_ms': 85, 'uptime_percent': 99.58}
[+149.820s] RC.RUNNER   : INFO     - Telus Agent CREATED check_network_status Node - ({'location': 'Burnaby'}, )
[+

In [8]:
viewer = AgentViewer(stamps=result.all_stamps,
                     node_heap=result.node_heap,
                     request_heap=result.request_heap)

viewer.display_graph()